# Branch and Bound Method
an algorithm to solve integer optimization problem using Branch and Bound method.

Name : Felix Fernando 

NIM : 10120082



In [3]:
import scipy 
import numpy as np 

def is_integer(n):
    if int(n) == n:
        return True
    else:
        return False

def branch_and_bound(c, A, b, integer_var, lb, ub, isMax = False, depth=1):
    # Optimal solution for LP Relaxation
    if(isMax):
        optimized = scipy.optimize.linprog(-c, A, b, method="highs", bounds=list(zip(lb, ub)))
    else:
        optimized = scipy.optimize.linprog(c, A, b, method="highs", bounds=list(zip(lb, ub)))

    x_candidate = optimized.x
    f_candidate = optimized.fun
    
    var_constraint_fulfilled = True

    if(f_candidate == None):
        if(isMax):
            return [], -np.inf, depth
        else:
            return [], np.inf, depth
            
    for idx, bool in enumerate(integer_var):
        if(bool and not is_integer(x_candidate[idx])):
            var_constraint_fulfilled = False 
            break
        else:
            var_constraint_fulfilled = True
        
    if(var_constraint_fulfilled): 
        if(isMax):
            return x_candidate, -f_candidate, depth
        else: 
            return x_candidate, f_candidate, depth
    else: 
        left_lb = np.copy(lb)
        left_ub = np.copy(ub)

        right_lb = np.copy(lb)
        right_ub = np.copy(ub)

        max_coeff_idx = -1
        for idx, val in enumerate(integer_var):
            if(val and not is_integer(x_candidate[idx])):
                if(max_coeff_idx == -1):
                    max_coeff_idx = idx
                elif(c[max_coeff_idx] < c[idx]):
                    max_coeff_idx = idx

        left_ub[max_coeff_idx] = np.floor(x_candidate[max_coeff_idx])
        right_lb[max_coeff_idx] = np.ceil(x_candidate[max_coeff_idx])

        x_left, f_left, depth_left = branch_and_bound(c, A, b, integer_var, left_lb, left_ub, isMax, depth + 1)
        x_right, f_right, depth_right = branch_and_bound(c, A, b, integer_var, right_lb, right_ub, isMax, depth + 1)

        if(isMax):
            if(f_left > f_right):
                return x_left, f_left, depth_left
            else: 
                return x_right, f_right, depth_right
        else:
            if(f_left < f_right):
                return x_left, f_left, depth_left
            else: 
                return x_right, f_right, depth_right

# Perform the integer optimization here

In [9]:
# Objective Function Coefficient
c = np.array([4, 3, 1])

# Constraint
A = np.array([[3, 2, 1], [2, 1, 2]])
b = np.array([7, 11])

# Select the integer var
integer_var = [False, True, True]

# Lower bound for each constraint 
lb = [0, 0, 0]
# Upper bound for each constraint 
ub = [None, None, None]

x_optimal, f_optimal, depth_optimal = branch_and_bound(c, A, b, integer_var, lb, ub, True)
print("========= Optimal Solutions =========")

print("Variable:")
for idx, var in enumerate(x_optimal): 
    print(f">  x{idx} = {var}")

print("Objective Function:")
print(f">  f = {f_optimal}")
print(f"Tree depth: {depth_optimal}")

========= Optimal Solutions =========
Variable:
>  x0 = 0.3333333333333333
>  x1 = 3.0
>  x2 = 0.0
Objective Function:
>  f = 10.333333333333334
Tree depth: 2
